## Fit of Neutron Reflectivity data from a polymer Brush

Suppose that we want to fit NR data of a polymer brushe at the Quartz/d-toluene interface. Polymer brushes have parabolic volume fraction profiles of the form: $\phi(z) = \phi(0) \left[ 1 - (\frac{z}{L})^n\right]$ where $\phi$ is the polymer volume fraction, $L$ is the brush extension, $n$ the exponent and $z$ the distance from the substrate.

First let's import _anaklasis_

In [ ]:
from anaklasis import anaklasis

We name the project as 'Brush_fit'. All results will be exported in a folder with this name. Then we import the experimental curve and we set input units to Angstrom.

In [ ]:
project='Brush_fit'
in_file=['DA34424a.dat']
units=['A'] # Angstrom units

then we set the details of the fit. Note that we will perform MCMC sampling.

In [ ]:
fit_mode=0 # linear
fit_weight=[1] 
method = 'mcmc' # Markov Chain Monte Carlo

We have a single laterally uniform layer on the substrate (brush) so a single defined model (patch) of 100% coverage

In [ ]:
patches=[1.0]

We define all layers except for the brush. Note that a thin water layer is always present on the substrate in these measurements. A global parameter (that will be defined later) is associated with the thickness of this thin water layer. all other layer parameters are fixed.

In [ ]:
system=[]
system.append([
	# Re_sld Im_sld thk rough solv description
	[  4.32e-6, 0.0, 0, 5, 0.0, 'Quartz'],
	[ -0.56e-6, 0.0, 'p3', 3, 0.0, 'water'],
	[  5.86e-6, 0.0,  0, 0, 1.0, 'd-toluene'],
	])

We will now insert between water layer and d-toluene, 20 slices representing the polymer brush. These layers will all have the same polymer sld and a varying solvent volume fraction ($1-\phi(z)$). For this reason we first define a symbolic expression for the solvent volume fraction.

In [ ]:
expr='1-p0+(p0/(p1**p2))*((p1/20)*(n-1.5))**p2'

where p0, p1, p2 are global parameters (we will define them later) representing $\phi(0)$, $L$ and the exponent. $n$ is the layer number that for our brush layer runs from 2 to 21. So the expression $(p1/20)*(n-1.5)$ gives the $z$ distance of each slice from the substrate. So now we insert the slices of the brush in the model _list_ using a _for loop_.

In [ ]:
for i in range(20): 
	system[0].insert(2,[ 1.41e-6, 0.0,'p1/20', 'p1/40', expr, 'layer'])

Note that we insert in system[0] since counting in python starts from zero. We start insering in position 2 since in position 1 we have the already defined thin water layer. All slices have the same sld, same thickness and we also apply a roughness (half slice thickness) to smooth the profile. The volume fraction profile is set according to the above defined symbolic expression.

Now we may define the global parameters. Note that the global parameters can be of type uniform or normal. For a uniform parameter we assume that the parameter is uniformly distributed in the given min/max bounds. For a normal parameter we assume that the parameter is normally distributed with the defined mean and standard deviation.

In [ ]:
global_param = [
    # param  min  max  description  type='uniform'
    # param  mean  sd  description  type='normal'
	['p0', 0.02, 0.20, 'phi0','uniform'],
	['p1', 100, 700, 'brush_length','uniform'],
	['p2', 1, 4, 'exponent','uniform'],
	['p3', 5, 2, 'water thickness','normal'],
	]

We have no multi-parameters or constraints to declare

In [ ]:
multi_param = [] # no multi-parameters
constraints = [] # no constraints

we also define the instrumental parameters. Background and scale are set as fixed.

In [ ]:
resolution=[0.06] # dQ/Q=6%
background = [[0.0,0.0,'uniform']] #background corrected data
scale = [[1.0,1.0,'uniform']] #data correctly scaled

We finally call the _anaklasis.fit_ function and store the results in the _tuple_ _res_. Note that we set _experror=True_ since our input data contain the $dR(Q)$ errors. 

In [ ]:
res = anaklasis.fit(project, in_file, units, fit_mode,fit_weight,method,resolution,patches, system,
global_param,multi_param, constraints,background,scale,experror=True, plot=True,fast=True)

_anaklasis_ automatically exports a number of plots for the reflectivity and profiles. A corner plot from the MCMC sampling is exported. If we want to plot manually the volume fraction profile and "zoom" close to the region of the variation of the brush profile we may use the results stored in the res _tuple_. 

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.ylim([0.0,0.102])
plt.plot(res[2][0][0][:,0],1-res[2][0][0][:,1],color='black')
plt.xlabel(r'$z(\AA)$')
plt.ylabel(r'$brush volume fraction$')
plt.draw()

For reference we include a detailed list of the contents of the _res_ _tuple_ in the current example:

	res[0][i][j][Q,R,R*Q^4] <- reflectivity list
	res[1][i][j][z,sld,sd sld] <- sld list
	res[2][i][j][z,solv, sd sld] <- solvent volume fraction list
	res[3][k][mean, sd] <- global_parameter fit list
	res[4][i][l][mean, sd] <- multi_parameter fit list
	res[5][i][mean, sd] <- background fit list
	res[6][i][mean, sd] <- scale fit list
	res[7][i][value] <- chi squared list

	where i the curve number starting from 0 (in the current example only one curve so 0)
	      j the model number starting from 0 (in the current example only one model so 0)
	      k the global_parameter number starting from 0 (in the current ecample from 0 to 3)
	      l the multi_parameter number starting from 0 (no multi_parameter so list is empty [])